In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from autogen_ext.models.openai import OpenAIChatCompletionClient

model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
   # api_key=os.environ.get("OPENAI_API_KEY")
)

In [3]:
from autogen_core.models import UserMessage

result = await model_client.create(
    [UserMessage(content="Tell me about Elon Musk in 2 bullet points", source="user")]
    )
print(result)

finish_reason='stop' content='- **Entrepreneurial Ventures:** Elon Musk is known for founding and leading several high-profile companies, including Tesla, Inc., which revolutionized the electric vehicle market, and SpaceX, which aims to reduce space transportation costs and enable Mars colonization.\n\n- **Innovative Technologies:** Musk is a proponent of advanced technologies such as Neuralink, which focuses on brain-machine interfaces, and The Boring Company, which targets urban transportation challenges through tunneling solutions.' usage=RequestUsage(prompt_tokens=18, completion_tokens=93) cached=False logprobs=None thought=None


In [4]:
# folder getter

import os

def get_project_structure(project_path, max_depth=3):
    """
    Recursively gets the structure of a project directory up to a given depth.

    Args:
        project_path (str): The path to the project directory.
        max_depth (int): Maximum depth to traverse.

    Returns:
        dict: A dictionary representation of the project structure.
    """
    def traverse(directory, depth):
        if depth > max_depth:
            return None  # Stop recursion at max depth
        
        structure = {}
        try:
            for item in os.listdir(directory):
                item_path = os.path.join(directory, item)
                if os.path.isdir(item_path):
                    structure[item] = traverse(item_path, depth + 1)
                else:
                    structure[item] = "file"
        except PermissionError:
            structure["error"] = "Permission Denied"
        
        return structure

    return {os.path.basename(project_path): traverse(project_path, 1)}

# Example usage
project_path = "C:\\Users\\vasanth.mu\\Desktop\\genai-training\\fin-5\\awesome-compose\\flask"  # Change to your actual project path
project_structure = get_project_structure(project_path)
print(project_structure)


{'flask': {'app': {'app.py': 'file', 'Dockerfile': 'file', 'requirements.txt': 'file'}, 'compose.yaml': 'file', 'README.md': 'file'}}
